In [34]:
import requests
from bs4 import BeautifulSoup

In [35]:
url = "https://www.aljazeera.com/where/mozambique/"
def getHTMLdocument(url):
    response = requests.get(url)
    return response.text

aljazeera_paper = getHTMLdocument(url)

In [36]:
base_url = "https://www.aljazeera.com"
soup = BeautifulSoup(aljazeera_paper, 'html.parser')
all_articles = soup.find_all("a", {"class": "u-clickable-card__link"})

article_links = [base_url + article['href'] for article in all_articles]
print(article_links)

['https://www.aljazeera.com/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again', 'https://www.aljazeera.com/news/2022/3/18/mozambique-cyclone-gombe-death-toll-rises-to-53', 'https://www.aljazeera.com/news/2022/3/4/mozambique-announces-new-prime-minister-and-finance-minister', 'https://www.aljazeera.com/economy/2022/3/1/analysis-can-african-gas-replace-russian-supplies-to-europe', 'https://www.aljazeera.com/news/2022/1/27/at-least-70-dead-from-tropical-storm-ana-in-southern-africa', 'https://www.aljazeera.com/news/2022/1/12/southern-africa-bloc-sadc-extends-mozambique-mission', 'https://www.aljazeera.com/program/start-here/2021/10/3/climate-change-and-famine-start-here', 'https://www.aljazeera.com/news/2021/9/24/in-mozambique-kagame-says-rwandan-troops-presence-to-continue', 'https://www.aljazeera.com/news/2021/8/8/rwanda-mozambique-forces-recapture-port-city-from-rebels', 'https://www.aljazeera.com/news/2021/7/10/rwanda-deploys-1000-soldiers-to-mozambique-cabo-delgado

In [37]:
from newspaper import Article
import json
from tqdm import tqdm

article_collections = []

# article_url = article_links[0]
# article = Article(article_url)
# article.download()
# article.parse()
# article_soup = BeautifulSoup(article.html, 'html.parser')
# article_soup.find("p", {"class": "article__subhead"}).find('em').string

for article_url in tqdm(article_links):
    article = Article(article_url)
    article.download()
    article.parse()
    article_soup = BeautifulSoup(article.html, 'html.parser')
    subtitle = ''
    try:
        subtitle =  article_soup.find("p", {"class": "article__subhead"}).find('em').string
    except:
        pass
    article_collections.append(
        { 'title': article.title, 'subtitle': subtitle, 'content': article.text }
    )

# print(len(article_collections))
with open('data.json', 'w') as f:
    json.dump(article_collections, f)

# json.dumps(article_collections)

100%|██████████| 14/14 [00:07<00:00,  1.89it/s]


# Sentiment Analysis

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [39]:
import json
with open('data.json') as f:
    data = json.load(f)

In [40]:
stopwords = nltk.corpus.stopwords.words("english")

In [41]:
import copy

cleaned_data = copy.deepcopy(data)
preprocessed_data = []
for each in cleaned_data:
    each['content'] = nltk.word_tokenize(each['content'])
    each['content'] = [w for w in each['content'] if w.lower() not in stopwords]
    each['content'] = [w for w in each['content'] if w.lower() not in [",", ".", "(", ")", "”", "[", "]", "“", "’", ":", "–"]]
    preprocessed_data.append(" ".join([w.lower() for w in each['content']]))

print(len(preprocessed_data))

14


## Sentiment Analysis for the Article Content only 

In [50]:
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean

sia = SentimentIntensityAnalyzer()
for each in data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each['content'])]
    print(mean(scores))
print("\n")
print("-------After Pre-processed Data-------")
print("\n")
for each in preprocessed_data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each)]
    print(mean(scores))

-0.15685625
-0.7474714285714286
-0.03154000000000001
0.18046666666666666
-0.360745
0.160475
-0.31245
-0.16955135135135135
0.03320000000000001
-0.2301764705882353
-0.319995
-0.1600921875
-0.389468
-0.03635


-------After Pre-processed Data-------


-0.68235
-0.9933
-0.304
0.5121142857142857
-0.9957
0.946
-0.6249
-0.4469
0.5859
-0.7179
-0.9949
-0.418925
-0.9894499999999999
-0.3142333333333333


## Sentiment Analysis using Title, Subtitle, and Content

In [59]:
content_score = []
for each in preprocessed_data:
    content_score.append([sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each)])

for each in data:  
    i = 0 
    title_score = max(sia.polarity_scores(article_collections[i]['title']), key = sia.polarity_scores(article_collections[i]['title']).get)
    subtitle_score = sia.polarity_scores(article_collections[i]['subtitle'])
    #total_score = (50 * mean(content_score[i])) + (25 * mean(title_score)) + (25 * mean(subtitle_score))
    print(title_score)
    i = i + 1

TypeError: 'value' is an invalid keyword argument for max()

# Visualization

In [47]:
import chart_studio.plotly as py
from plotly.graph_objs import *